# Robust Lookup Table: AI Model

The goal of this notebook is to explore and experiment with a deep learning approach to create the Robust Lookup Table.

### Summary

* #### I. Generate fake scenarios & backends
* #### II. Design Deep learning model
* #### II. Train model


## I. Generate fake scenarios & backends

### I.1. Generate fake Scenarios


In [2]:
import random
import uuid
import hashlib
import numpy as np

In [149]:
# -- Fake scenarios

# ScenarioGeneratorConfig
# - size: the fixed size of the lookup table.
# - nBeforeBounds(x, y): nBefore ∈ [x, y].
# - nAfterBounds(x, y): nAfter ∈ [x, y].
# - variance(x, y): x < min(nBefore,nAfter)/max(nBefore,nAfter); y < max(nBefore,nAfter) - min(nBefore,nAfter)
# # - sizeBounds(x, y): lookup table size ∈ [x, y].
class ScenarioGeneratorConfig:
    size: int
    nBeforeBounds: (int, int)
    nAfterBounds: (int, int)
    variance: (float, int)
    # sizeBounds: (int, int)

    def __init__(
        self,
        size: int,
        nBeforeBounds: (int, int),
        nAfterBounds: (int, int),
        variance: (float, int),
        # sizeBounds: (int, int),
    ):
        if nBeforeBounds[1] > size or nAfterBounds[1] > size:
            raise Exception("nBeforeBounds and nAfterBounds cannot exceed size")

        self.size = size
        self.nBeforeBounds = nBeforeBounds
        self.nAfterBounds = nAfterBounds
        self.variance = variance
        # self.sizeBounds = sizeBounds

class Scenario:
    nBefore: int
    nAfter: int
    size: int

def validate_scenario(cfg: ScenarioGeneratorConfig, scenario: Scenario) -> bool:
    var = cfg.variance[0]
    delta = cfg.variance[1]

    _min = min([scenario.nBefore, scenario.nAfter])
    _max = max([scenario.nBefore, scenario.nAfter])
    _var = _min/_max
    _delta = _max - _min
    _sz = scenario.size

    return _var <= var and _delta <= delta and _max <= _sz and _min != _max

# creates a new scenario generator.
def new_scenario_generator(cfg):
    while True:
        scenario = Scenario()
        scenario.nBefore = random.randint(cfg.nBeforeBounds[0], cfg.nBeforeBounds[1])
        scenario.nAfter = random.randint(cfg.nAfterBounds[0], cfg.nAfterBounds[1])
        scenario.size = cfg.size
        # scenario.size = random.randint(cfg.sizeBounds[0], cfg.sizeBounds[1])

        if validate_scenario(cfg, scenario):
            yield scenario

In [148]:
nBeforeBounds = (3, 47)
nAfterBounds = (1, 47)
variance = (1.0, 10)
size = 47

cfg = ScenarioGeneratorConfig(size, nBeforeBounds, nAfterBounds, variance)
sc = new_scenario_generator(cfg)

for i in range(3):
    s = next(sc)
    print(s.__dict__)

{'nBefore': 40, 'nAfter': 41, 'size': 47}
{'nBefore': 29, 'nAfter': 37, 'size': 47}
{'nBefore': 29, 'nAfter': 39, 'size': 47}


### I.2. Generate fake Backends

In [150]:
# new_backend returns a dict representing a backend
# k,v:
# - id,uuid
# - hash,int
def new_backend():
    id = uuid.uuid4()
    _h = hashlib.sha256()
    _h.update(id.bytes_le)
    _b = _h.digest()
    h = int.from_bytes(_b[24:32], "little")
    
    return (id, h)

def new_backend_list_permutation(l: list, size: int) -> list:
    _p = np.random.permutation(l)
    return _p[0:size]

# new_backend_generator takes a list of Scenario and yields a tuple of 2 lists of Backend.
# The "before" list and the "after" list.
def new_backend_generator(scenarioGenerator):
    while True:
        sc = next(scenarioGenerator)
        _min = min([sc.nBefore, sc.nAfter])
        _max = max([sc.nBefore, sc.nAfter])
        isMinBefore = sc.nBefore < sc.nAfter
        l_min = []
        l_max = []

        # create the l_max backend array.
        for i in range(_max):
            b = new_backend()
            l_max.append(b)
            if len(l_min) < _min:
                l_min.append(b)

        # create l_min array by randomly choosing _min elements of l_max.
        l_min = new_backend_list_permutation(l_max, _min)

        # sort both arrays.
        l_max = sorted(l_max, key=lambda x: str(x[0]))
        l_min = sorted(l_min, key=lambda x: str(x[0]))
        
        if isMinBefore:
            yield (l_min, l_max)
        else:
            # print([x.__dict__ for x in l_min])
            yield (l_max, l_min)

In [151]:
generator = new_backend_generator(new_scenario_generator(cfg))

for i in range(2):
    t = next(generator)
    print(f"- Before: len={len(t[0])} example_value={t[0][0]}")
    print(f"- After: len={len(t[1])} example_value={t[1][0]}")

- Before: len=25 example_value=[UUID('087b90d2-0195-47f7-9373-26f03bf00be4') 11470088727391372139]
- After: len=35 example_value=(UUID('008f5c09-a190-4506-a9af-beb169a7de4e'), 13543712438097616289)
- Before: len=39 example_value=(UUID('04bd519f-f052-40ff-83b6-3739ea0ef491'), 16152123668707616722)
- After: len=30 example_value=[UUID('04bd519f-f052-40ff-83b6-3739ea0ef491') 16152123668707616722]


## II. Design Deep learning model

### Definitions

- Let `m` equal to the length of the lookup table: `m=len(lookup_table)`.
- Let `input` an array of length equal to m: `len(input)=m`. 
  - Each entry in `input` represents a backend.
- Let `h(i)` the hash of the i-th backend in `input`.
- Let `input[i]=h(i) % m`.
- Let `n` the number of backend actually represented in `input`.
  - Because: `nAfter != nBefore` and `max(nAfter,nBefore) <= m`.
- Let `output` a matrix of size `m*m`.
- Let `output[j]` the j-th row in the `output` matrix.
  - The j-th row of the `output` matrix represents the j-th entry of
     the lookup table.
- Let `o(i,j)` the i-th entry in `output[j]`.
  - `o(i,j)` is the probability of the i-th backend being mapped to the
     j-th entry of the lookup table.

### Input data

Problem 1: How to represent data in the inputs that is unmapped? 
- E.g. if the modulo of a hash is equal to 0, how should we represent entries
  that are out of bound.
- In other words, if `n=13` and `m=47`, how do we represent the entries with
  index in the range of [13:47]?

Definitions:
- Let `in-bound` entries the name of entries in the range [0:13].
- Let `out-of-bound` entries the name of entries in the range [13:47].

Solution:
- If we normalize `in-bound` entries as real numbers in [0,1], then we can
set `out-of-bound` entries to `-1`.
- Another solution would be to represent the input as a `m*m` matrix. 
  - The i-th row representing the i-th backend.
  - The j-th entry in i-th row representing the modulo of the hash of the i-th backend
  - If the j-th entry of the i-th row is equal to 1, it means 
  - If all entries of the i-th row are equal to 0, then it means there are no backends 
    there.

Problem 2: what if multiple backend have the same modulo?
- This is particularly problematic if 2 subsequent backends resolves to the same modulo
  and 1 of the backend becomes down. 
- In that case, there is not way to identify which backend was dropped from the model's
  point of view. 
- Hence there is a 50% chance to reaffect packets away from a healthy backend.

Solution:
- Compute multiple hash for each backend. Or split the 128-bit hash into 4 int32 and 
  compute 4 modulo. The probability of encountering 4 collisions in the same order
  would be significantly lower (the actual improvement has not be calculated).

### Model training:

- Pass the "before" training data through the model.
- Pass the "after" training data through the model.
- Compute "even distribution" score: to ensure the backends are evenly
  distributed in the output.
- Compute "validity" score: 
  - to ensure the model does not make inference `out-of-bound`.
- Optional: compute a "confidence score", by calculating how likely the top inference
  is compared to other o(i,j) value in i-th row.
- Compute the % of unchanged entries between "Before" and "After".
- Compute the stability score.
- Compute loss function from "validity", "even distribution" and "stability" score.


In [152]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [162]:
# NN takes `m` (int) as a paremeter. 
# `m` is the length of the lookup table.
# input dimensions is a tensor of size `m` and dimension 1.
# output dimensions are `m*m` matrices.
class NN(nn.Module):
    size: int

    def __init__(self, m: int):
        super().__init__()

        self.size = m
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(m, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, m*m),
        )

    def forward(self, x) -> (list, list):
        _bef_in, _aft_in = x

        # -- prepare
        bef_in = self.clean_input(_bef_in)
        aft_in = self.clean_input(_aft_in)

        # -- infer
        _bef_out = self.forward_once(bef_in)
        _bef_aft = self.forward_once(aft_in)

        # -- post-process
        bef_out = self.clean_output(_bef_in, _bef_out)
        aft_out = self.clean_output(_aft_in, _aft_out)
        
        return bef_out, aft_out

    def forward_once(self, x):
        # -- debug
        print("x:", x)
        # x = self.flatten(x)
        return self.linear_relu_stack(x)

    def clean_input(self, x):
        out = torch.full((size,), -1)
        for i, backend in enumerate(x):
            # backend[1] is the hash.
            out[i] = backend[1] % self.size
        return out

    def clean_output(self, x_in, x_out):
        # We want to return a list of length `m` that
        # associate each i-th entry with a backend uuid.
        # 
        # An entry at index `i` is obtained by fetching the
        # uuid of the backend at index `j` of `x_in`.
        # `j` is the index of the highest value of the j-th
        # row of `x_out`.
        # 
        # - x_in is the raw input.
        # - x_out are matrices of size m*m.
        out = []
        for row in x_out:
            # the first element is the max value.
            # we may want to output it in order to calculate the loss.
            # this would measure the confidence of the algorithm in the inference. 
            _, j = torch.max(row, 0) 
            out.append(str(x_in[j][0]))
        return out

In [155]:
x = torch.rand(5)
y = [ 0, 1, 2, 3, 4 ]
print(x)
print(torch.max(x, 0)[1])
print(y[torch.max(x, 0)[1]])

out = torch.full((size,), -1)
out.size()

tensor([0.8188, 0.7926, 0.7015, 0.2051, 0.6772])
tensor(0)
0


torch.Size([47])

In [164]:
model = NN(47).to(device)
print(model(next(generator)))

x: tensor([ 2, 29, 21, 23, 39, 11, 17,  7, 16, 41, 25, 10, 18, 22,  4, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float

In [49]:
class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()
        # self.parameter = nn.Parameter(torch.tensor(some_parameter)) # Example learnable parameter

    def forward(self, inputs, outputs):
        before, after = inputs
        weight = torch.abs(target) + self.parameter
        loss = torch.mean(weight * (output - target)**2)
        return loss

Loss().forward((0, 1), 0)

0 1


NameError: name 'target' is not defined

## III. Train the model

In [45]:
# Algorithm parameters
m = 47
nBeforeBounds = (3, m)
nAfterBounds = (1, m)
variance = (1.0, 10)

# Hyperparameters
learning_rate = 0.001
epochs = 10
batch_size = 64

In [50]:
cfg = ScenarioGeneratorConfig(m, nBeforeBounds, nAfterBounds, variance)
backend_generator = new_backend_generator(new_scenario_generator(cfg))

model = NN(m).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = Loss()

for epoch in range(epochs):
    for i in range(batch_size):
        # -- reset optimizer
        optimizer.zero_grad()

        # -- generate inputs
        inputs = next(backend_generator)

        # -- run model
        outputs = model(inputs)

        # -- compute loss
        loss = loss_fn(inputs, outputs)

        #        
        loss.backward()
        optimizer.step()
        

AttributeError: 'tuple' object has no attribute 'flatten'